In [1]:
from turtle import st
import transbigdata as tbd
import networkx as nx
import pandas as pd
import numpy as np
from shapely.geometry import LineString
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
import ast
from multiprocessing import Pool
from itertools import islice
import math

warnings.filterwarnings("ignore")
plt.rcParams['font.sans-serif'] = 'SimSun'  # 用来正常显示中文
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

line = pd.read_pickle('line.pkl')
stop = pd.read_pickle('stop.pkl')

#调整站名和地铁线路名
stop.loc[stop['line'] == '地铁4号线内圈', 'line'] = '地铁4号线'
stop.loc[stop['line'] == '地铁4号线外圈', 'line'] = '地铁4号线'
line.loc[line['line'] == '地铁4号线内圈', 'line'] = '地铁4号线'
line.loc[line['line'] == '地铁4号线外圈', 'line'] = '地铁4号线'
stop.loc[stop['line'] == '地铁2号线西延线', 'line'] = '地铁2号线'
line.loc[line['line'] == '地铁2号线西延线', 'line'] = '地铁2号线'
stop.loc[stop['linename'] == '地铁2号线西延线(徐泾东-蟠祥路)', 'linename'] = '地铁2号线(徐泾东-蟠祥路)'
stop.loc[stop['linename'] == '地铁2号线西延线(蟠祥路-徐泾东)', 'linename'] = '地铁2号线(蟠祥路-徐泾东)'
line.loc[line['linename'] == '地铁2号线西延线(徐泾东-蟠祥路)', 'linename'] = '地铁2号线(徐泾东-蟠祥路)'
line.loc[line['linename'] == '地铁2号线西延线(蟠祥路-徐泾东)', 'linename'] = '地铁2号线(蟠祥路-徐泾东)'
stop.loc[stop['line'] == '地铁13号线西延伸', 'line'] = '地铁13号线'
line.loc[line['line'] == '地铁13号线西延伸', 'line'] = '地铁13号线'
stop.loc[stop['linename'] == '地铁13号线西延伸(金运路-诸光路)', 'linename'] = '地铁13号线(金运路-诸光路)'
stop.loc[stop['linename'] == '地铁13号线西延伸(诸光路-金运路)', 'linename'] = '地铁13号线(诸光路-金运路)'
line.loc[line['linename'] == '地铁13号线西延伸(金运路-诸光路)', 'linename'] = '地铁13号线(金运路-诸光路)'
line.loc[line['linename'] == '地铁13号线西延伸(诸光路-金运路)', 'linename'] = '地铁13号线(诸光路-金运路)'
stop.loc[stop['line'] == '地铁17号线西延线', 'line'] = '地铁17号线'
line.loc[line['line'] == '地铁17号线西延线', 'line'] = '地铁17号线'
stop.loc[stop['linename'] == '地铁17号线西延线(西岑-东方绿舟)', 'linename'] = '地铁17号线(西岑-东方绿舟)'
stop.loc[stop['linename'] == '地铁17号线西延线(东方绿舟-西岑)', 'linename'] = '地铁17号线(东方绿舟-西岑)'
line.loc[line['linename'] == '地铁17号线西延线(西岑-东方绿舟)', 'linename'] = '地铁17号线(西岑-东方绿舟)'
line.loc[line['linename'] == '地铁17号线西延线(东方绿舟-西岑)', 'linename'] = '地铁17号线(东方绿舟-西岑)'
stop.loc[stop['line'] == '地铁18号线二期', 'line'] = '地铁18号线'
line.loc[line['line'] == '地铁18号线二期', 'line'] = '地铁18号线'
stop.loc[stop['linename'] == '地铁18号线二期(大康路-长江南路)', 'linename'] = '地铁18号线(大康路-长江南路)'
stop.loc[stop['linename'] == '地铁18号线二期(长江南路-大康路)', 'linename'] = '地铁18号线(长江南路-大康路)'
line.loc[line['linename'] == '地铁18号线二期(大康路-长江南路)', 'linename'] = '地铁18号线(大康路-长江南路)'
line.loc[line['linename'] == '地铁18号线二期(长江南路-大康路)', 'linename'] = '地铁18号线(长江南路-大康路)'


#构建地铁网络拓扑图
def split_subwayline(line, stop):
    def getline(r2, line_geometry):
        ls = []
        if r2['o_project'] <= r2['d_project']:
            tmp1 = np.linspace(r2['o_project'], r2['d_project'], 10)
        if r2['o_project'] > r2['d_project']:
            tmp1 = np.linspace(  # pragma: no cover
                r2['o_project'] - line_geometry.length, r2['d_project'], 10)
            tmp1[tmp1 < 0] = tmp1[tmp1 < 0] + line_geometry.length  # pragma: no cover
        for j in tmp1:
            ls.append(line_geometry.interpolate(j))
        return LineString(ls)

    lss = []
    for k in range(len(line)):
        r = line.iloc[k]
        line_geometry = r['geometry']
        tmp = stop[stop['linename'] == r['linename']].copy()
        for i in tmp.columns:
            tmp[i + '1'] = tmp[i].shift(-1)
        tmp = tmp.iloc[:-1]
        tmp = tmp[['stationnames', 'stationnames1',
                   'geometry', 'geometry1', 'linename', 'line']]
        tmp['o_project'] = tmp['geometry'].apply(
            r['geometry'].project)
        tmp['d_project'] = tmp['geometry1'].apply(
            r['geometry'].project)
        tmp['geometry'] = tmp.apply(
            lambda r2: getline(r2, line_geometry), axis=1)
        lss.append(tmp)
    metro_line_splited = pd.concat(lss).drop('geometry1', axis=1)
    metro_line_splited.crs = 'epsg:4326'
    metro_line_splited['length'] = metro_line_splited.to_crs(epsg=3857).length
    metro_line_splited = metro_line_splited.drop(
        ['o_project', 'd_project'], axis=1)
    return metro_line_splited


def metro_network(line, stop, transfertime=5, nxgraph=True):
    # Obtain edge1: Network edge for line section.
    linestop = stop.copy()
    if ('speed' not in line.columns) | ('stoptime' not in line.columns):
        raise ValueError(  # pragma: no cover
            'Lines should have `line` column to store line name,'
            '`speed` column to store metro speed and'
            '`stoptime` column to store stop time at each station'
        )
    for i in linestop.columns:
        linestop[i + '1'] = linestop[i].shift(-1)
    linestop = linestop[linestop['linename'] == linestop['linename1']].copy()
    linestop = linestop.rename(
        columns={'stationnames': 'ostop', 'stationnames1': 'dstop'})
    linestop['ostation'] = linestop['line'] + linestop['ostop']
    linestop['dstation'] = linestop['line'] + linestop['dstop']
    edge1 = linestop[['ostation', 'dstation']].copy()

    # calculate travel time for edge1
    # calculate distance
    metrolinesplit = split_subwayline(line, stop)
    metrolinesplit['ostation'] = metrolinesplit['line'] + \
                                 metrolinesplit['stationnames']
    metrolinesplit['dstation'] = metrolinesplit['line'] + \
                                 metrolinesplit['stationnames1']
    metrolinesplit = metrolinesplit[['ostation', 'dstation', 'line', 'length']]
    edge1 = pd.merge(edge1, metrolinesplit, how='left')
    edge1 = pd.merge(edge1, line[['line', 'speed', 'stoptime']])

    # calculate duration
    edge1['duration'] = 60 * (edge1['length'] / 1000) / \
                        edge1['speed'] + edge1['stoptime']
    edge1 = edge1[['ostation', 'dstation', 'duration']].drop_duplicates(
        subset=['ostation', 'dstation'])

    # Obtain edge2: Network edge for transfering.
    linestop = stop.copy()
    linestop['station'] = linestop['line'] + linestop['stationnames']
    tmp = linestop.groupby(['stationnames'])[
        'linename'].count().rename('count').reset_index()
    tmp = pd.merge(linestop, tmp[tmp['count'] > 2]
    ['stationnames'], on='stationnames')
    tmp = tmp[['stationnames', 'line', 'station']].drop_duplicates()
    tmp = pd.merge(tmp, tmp, on='stationnames')

    edge2 = tmp[tmp['line_x'] != tmp['line_y']][['station_x', 'station_y']]
    # All transfer time are set as the same, export `edge2` for further degign
    edge2['duration'] = transfertime
    edge2.columns = edge1.columns
    edge = pd.concat([edge1, edge2])
    node = list(edge['ostation'].drop_duplicates())
    if nxgraph:
        import networkx as nx
        G = nx.Graph()
        G.add_nodes_from(node)
        G.add_weighted_edges_from(edge.values)
        return G
    else:
        return edge1, edge2, node  # pragma: no cover


line['speed'] = 37.3  #地铁运行速度 km/h
line['stoptime'] = 0.3  #停站时间 min

G = metro_network(line, stop, transfertime=5, nxgraph=True)

In [2]:
a = split_subwayline(line, stop)

In [5]:
stop['ostation'] = stop['line'] + stop['stationnames']

In [8]:
stop.to_csv('stop.csv', index=False, encoding='gbk')

NameError: name 'line' is not defined

In [2]:
# K短路算法
def get_k_shortest_paths(G, lineostation, linedstation, k):
    try:
        paths = islice(nx.shortest_simple_paths(G, lineostation, linedstation, weight='weight'), k)
        return list(paths)
    except nx.NetworkXNoPath:
        # 如果没有路径，返回一个空列表或者特定的标记
        return []
    
def get_shortest_paths(G, ostation,dstation):
    try:
        # 尝试获取k条最短路径
        path = nx.shortest_path(G, ostation, dstation)
        # 确保返回的路径数量不超过k
        return path
    except nx.NetworkXNoPath:
        # 如果没有路径，返回一个空列表或者特定的标记
        return []
    except nx.NodeNotFound:
        # 如果OD站不在图内，返回空列表
        return []
#计算路径的在车时间
def get_path_traveltime(G, path):
    traveltime = 0
    for i in range(len(path) - 1):
        traveltime += G.get_edge_data(path[i], path[i + 1])['weight']
    return traveltime

# 计算换乘次数 
def get_transfer_count(path):
    transfer_count = 0
    for i in range(1, len(path)):
        # 如果当前站点和前一个站点的线路不同，则换乘次数加一
        if path[i].split('线')[0] != path[i-1].split('线')[0]:
            transfer_count += 1
    return transfer_count

def verify_path(paths):
    dropindex=[]
    for j in range(len(paths)):        
        path = paths[j]
#计算换乘次数和去除无效路径
        used_lines = set()
        used_lines.add(path[0].split('线')[0])
        repeated_lines = set()
        for s in range(len(path)-1):
            if path[s].split('线')[0] != path[s+1].split('线')[0] :#判断是否换乘
                if path[s+1].split('线')[0] in used_lines:# 检查线路是否已经使用过
                    repeated_lines.add(path[s+1].split('线')[0])# 如果已使用过，记录为重复线路
                else:
                    used_lines.add(path[s+1].split('线')[0])# 如果是第一次使用，添加到已使用的线路集合中
        transfercount = len(used_lines)-1
        if len(repeated_lines) != 0 or transfercount >=5: #删除有重复线路的路径\换乘次数大于5的路径
            dropindex.append(j)
    paths = [element for index, element in enumerate(paths) if index not in dropindex]
    return paths
#更新客流表
def update_flow(ls,passenger_flow):
    ls = pd.concat(ls)
    flow = ls.groupby(['o','d'])['volume'].sum().reset_index()
    flow = pd.merge(passenger_flow,flow ,on=['o','d'],suffixes=('_df1', '_df2'),how='left')
    flow['volume_df2'].fillna(0,inplace=True)
    flow['volume']=flow['volume_df2']+flow['volume_df1']
    flow.drop(columns=['volume_df1','volume_df2'],inplace=True)
    return flow

In [4]:
#读取数据{OD表（'ostation','dstation','volume')，客流表('o','d','volume')}
#读取OD
OD = pd.read_csv('OD_onepath.csv')

#读取客流表
flow = pd.read_csv('flow.csv')

stations = pd.read_excel('stations.xlsx')
stations = stations['station']
stations_loss = pd.read_excel('stations.xlsx')
stations_loss['losstime'] = 0
#需要的参数
#拥挤度
alpha=1.1
beta=1.5
#出行阻抗
metrogap=3 #地铁发车间隔
capacity=(60/metrogap)*310*6#一小时内区间满载定员
transfertime=5 #换乘时间
intime = 5 #进站时间
outtime = 5 #出站时间
#logit模型
theta = 2.18
sumfee=0#乘客广义出行费用之和
#损失时间参数
pausedtime = 30

In [5]:
#识别包含该区间的OD
def recognaze_section(OD,ostation,dstation):
    index=set()
    r=OD['paths']
    for i in range(len(r)):
        if ostation in r[i] and dstation in r[i]:
            index.add(i)
    index=list(index)
    return OD.iloc[index].reset_index(drop=True)

#识别包含车站的OD
def recognaze_station(OD,station):
    index=set()
    r=OD['paths']
    for i in range(len(r)):
        if station in r[i]:
            index.add(i) 
    index=list(index)
    return OD.iloc[index].reset_index(drop=True)

def get_congestion(path,passenger_flow):    
    tmp = pd.DataFrame(path,columns=['o'])
    tmp['d'] = tmp['o'].shift(-1)
    tmp = tmp.iloc[:-1]
    flow = pd.merge(tmp,passenger_flow,on=['o','d'],how='left')['volume'].sum()
    rate = flow/(capacity*len(path))
    if rate <= 0.7:
        congestion = 0
    elif rate <= 1:
        congestion = alpha*(rate-0.7)
    else:
        congestion = alpha*(1-0.7)+beta*(rate-1)
    return congestion
def get_cost(path,congestion):
    transfercount = get_transfer_count(path)
    f1 = get_path_traveltime(G,path) - transfercount*transfertime #在车时间
    f2 = transfercount*transfertime #换乘时间
    f3 = metrogap*0.5+intime+outtime #进出站时间加候车时间
    fee = f1*(1+congestion) + f2 + f3
    return fee

In [6]:
OD['paths'] = OD['paths'].apply(ast.literal_eval)

,o,d,volume
0,地铁1号线莘庄,地铁1号线外环路,2285.0
1,地铁1号线外环路,地铁1号线莲花路,2157.0
2,地铁1号线莲花路,地铁1号线锦江乐园,2541.0
3,地铁1号线锦江乐园,地铁1号线上海南站,2488.0
4,地铁1号线上海南站,地铁1号线漕宝路,2612.0
...,...,...,...
1295,磁浮线龙阳路,地铁7号线龙阳路,736.0
1296,磁浮线龙阳路,地铁16号线龙阳路,1977.0
1297,磁浮线龙阳路,地铁18号线龙阳路,336.0
1298,磁浮线浦东国际机场,地铁2号线浦东国际机场,0.0


In [21]:
#计算车站的客流损失时间
tqdm.pandas()
for i in tqdm(range(len(stations))):
    losstime = 0
    OD_affected = recognaze_station(OD, stations[i])
    if OD_affected.empty:
        continue
    G_station = G.copy()
    G_station.remove_node(stations[i])
    OD_affected['path'] = OD_affected.apply(lambda x: get_shortest_paths(G_station, x['ostation'], x['dstation']),axis=1)
    OD_affected['path'] = OD_affected.apply(lambda x :verify_path(x['path']), axis=1)
    nopath = OD_affected['path'].apply(lambda x: isinstance(x, list) and not x)
    num_nopath =nopath.sum()
    losstime = OD_affected[nopath]['volume'].sum() * pausedtime
    OD_alter = OD_affected[~nopath].reset_index(drop=True)
    OD_alter['paths'] = OD_alter['paths'].apply(ast.literal_eval)
    altertime = 0
    for j in range(len(OD_alter)):
        r = OD_alter.iloc[j]
        path_old = r['paths']
        path_alter = r['path']
        congestion_old = get_congestion(path_old,flow)
        congestion_alter = get_congestion(path_alter,flow)
    #计算出行阻抗
        cost_old = get_cost(path_old,congestion_old)
        cost_alter = get_cost(path_alter,congestion_alter)
        altertime =altertime + (cost_alter-cost_old)*r['volume']
    losstime = losstime + altertime
    stations_loss['losstime'][i] = losstime

100%|██████████| 530/530 [06:09<00:00,  1.43it/s]


In [22]:
stations_loss

,station,losstime
0,地铁1号线莘庄,1069950.0
1,地铁1号线外环路,1125630.0
2,地铁1号线莲花路,1375230.0
3,地铁1号线锦江乐园,1467540.0
4,地铁1号线上海南站,1773180.0
...,...,...
525,浦江线三鲁公路,5850.0
526,浦江线闵瑞路,5850.0
527,浦江线浦航路,5850.0
528,浦江线东城一路,5850.0


In [23]:
stations_loss.to_csv('stations_losstime.csv',index=False)

In [25]:
stations_loss.to_csv('站点客流损失时间.csv',index=False,encoding='gbk')

NameError: name 'flow' is not defined

In [64]:
stations_loss.to_csv('stations_loss2.csv',index=False)

In [7]:
sections = pd.read_excel('sections.xlsx')
sections_loss = pd.read_excel('sections.xlsx')
sections_loss['losstime'] = 0

In [10]:
#计算区间客流损失时间
for i in tqdm(range(len(sections))):
    losstime = 0
    h = sections.iloc[i]
    o = h['o']
    d = h['d']
    OD_affected = recognaze_section(OD, o , d)
    if OD_affected.empty:
        continue
    G_station = G.copy()
    G_station.remove_edge(o,d)
    OD_affected['path'] = OD_affected.apply(lambda x: get_shortest_paths(G_station, x['ostation'], x['dstation']),axis=1)
    OD_affected['path'] = OD_affected.apply(lambda x :verify_path(x['path']), axis=1)
    nopath = OD_affected['path'].apply(lambda x: isinstance(x, list) and not x)
    num_nopath =nopath.sum()
    losstime = OD_affected[nopath]['volume'].sum() * pausedtime
    OD_alter = OD_affected[~nopath].reset_index(drop=True)
    altertime = 0
    for j in range(len(OD_alter)):
        r = OD_alter.iloc[j]
        path_old = r['paths']
        path_alter = r['path']
        congestion_old = get_congestion(path_old,flow)
        congestion_alter = get_congestion(path_alter,flow)
    #计算出行阻抗
        cost_old = get_cost(path_old,congestion_old)
        cost_alter = get_cost(path_alter,congestion_alter)
        altertime =altertime + (cost_alter-cost_old)*r['volume']
    losstime = losstime + altertime
    sections_loss['losstime'][i] = losstime

100%|██████████| 1020/1020 [12:22<00:00,  1.37it/s]


In [12]:
sections_loss.to_csv('sections_losstime.csv',index=False)

In [13]:
sections_loss.to_csv('区间客流损失时间.csv',index=False,encoding='gbk')